In [1]:
!pip install PyMuPDF Pillow pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 63.1 MB/s eta 0:00:00


In [14]:
!pip install tesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=d1bf43086cef963a95b66757864f38687c0d7665b8bf3d48cc948df373efe269
  Stored in directory: /root/.cache/pip/wheels/71/c9/aa/698c579693e83fdda9ad6d6f0d8f61ed986e27925ef576f109
Successfully built tesseract


In [16]:
!apt-get install -y tesseract-ocr
!pip install pytesseract PyMuPDF pillow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,292 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123614 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [19]:
import fitz  # PyMuPDF
import os
import re
import io
from PIL import Image
import pytesseract

def clean_text(text):
    # Remove special characters and excessive spaces
    cleaned = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    cleaned = re.sub(r'[^\w\s]', '', cleaned)  # Remove special characters
    return cleaned.strip()

def extract_text_from_image(image):
    return pytesseract.image_to_string(image)

def clean_filename(filename):
    # Remove numbers and special characters from the filename
    cleaned = re.sub(r'[0-9_-]', '', filename)  # Remove numbers, underscores, and dashes
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()  # Replace multiple spaces with a single space
    return cleaned + "_cleaned.pdf"

def process_pdf(input_pdf_path, output_folder):
    doc = fitz.open(input_pdf_path)
    all_text = []

    for page in doc:
        # Extract text from the page
        text = page.get_text()
        all_text.append(clean_text(text))

        # Extract images and perform OCR
        image_list = page.get_images(full=True)  # Get all images on the page
        for img_index, img in enumerate(image_list):
            xref = img[0]  # Image xref
            base_image = doc.extract_image(xref)  # Extract the image
            image_bytes = base_image["image"]  # Get the image bytes
            image = Image.open(io.BytesIO(image_bytes))  # Open image from bytes
            ocr_text = extract_text_from_image(image)
            all_text.append(clean_text(ocr_text))

    all_text = "\n".join(all_text).strip()  # Combine all cleaned text

    # Prepare output PDF
    output_pdf_name = clean_filename(os.path.basename(input_pdf_path))  # Clean the filename
    output_pdf_path = os.path.join(output_folder, output_pdf_name)

    # Create a new PDF to save the text
    cleaned_doc = fitz.open()

    # Define A4 page dimensions and margins
    page_width = 595  # A4 width in points
    page_height = 842  # A4 height in points
    margin = 72  # 1 inch margin

    # Split the text into smaller blocks to fit on the page
    text_blocks = all_text.split('\n')

    # Create the first page
    cleaned_page = cleaned_doc.new_page(width=page_width, height=page_height)

    text_area_height = page_height - 2 * margin  # Usable height on the page
    text_box = fitz.Rect(margin, margin, page_width - margin, page_height - margin)
    text_to_insert = ""

    for block in text_blocks:
        temp_insert = text_to_insert + block + "\n"

        # Test if the current block fits within the page's text area
        if cleaned_page.insert_textbox(text_box, temp_insert, fontsize=12, color=(0, 0, 0)) <= 0:
            # If the text doesn't fit, finalize the current page and start a new one
            cleaned_page.insert_textbox(text_box, text_to_insert, fontsize=12, color=(0, 0, 0))
            cleaned_page = cleaned_doc.new_page(width=page_width, height=page_height)  # New page
            text_to_insert = block + "\n"  # Start new page with current block
        else:
            text_to_insert = temp_insert  # Add block to current page

    # Insert any remaining text on the last page
    if text_to_insert.strip():
        cleaned_page.insert_textbox(text_box, text_to_insert, fontsize=12, color=(0, 0, 0))

    cleaned_doc.save(output_pdf_path)
    cleaned_doc.close()
    doc.close()

# Specify input directory and output folder
input_folder = '/content/data'  # Replace with your input folder path
output_folder = '/content/cleanedData'  # Replace with your output folder path

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process all PDF files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.pdf'):
        process_pdf(os.path.join(input_folder, filename), output_folder)

print("Processing complete. Cleaned PDFs saved in:", output_folder)


Processing complete. Cleaned PDFs saved in: /content/cleanedData
